In [ ]:
#Imports

import numpy as np
import tensorflow as tf
import time, random
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter("ignore", DeprecationWarning)
warnings.simplefilter("ignore", FutureWarning)

import scipy.stats as ss

In [ ]:
#Parameters

def get_weights(name,shape):
    with tf.variable_scope("weights", reuse=tf.AUTO_REUSE): 
        return tf.get_variable(name=name,shape=shape,initializer = tf.contrib.layers.xavier_initializer(uniform=False),regularizer = tf.contrib.layers.l2_regularizer(tf.constant(0.001, dtype=tf.float32)))
    
def get_bias(name,shape):
    with tf.variable_scope("bias", reuse=tf.AUTO_REUSE):
        return tf.get_variable(name=name,shape=shape,initializer = tf.zeros_initializer())


In [ ]:
#Layers

def conv2d(inp,name,kshape,s):
    with tf.variable_scope(name) as scope:
        kernel = get_weights('weights',shape=kshape)
        conv = tf.nn.conv2d(inp,kernel,[1,s,s,1],'SAME')
        bias = get_bias('biases',shape=kshape[3])
        preact = tf.nn.bias_add(conv,bias)
        convlayer = tf.nn.relu(preact,name=scope.name)
    return convlayer

def maxpool(inp,name,k,s):
    return tf.nn.max_pool(inp,ksize=[1,k,k,1],strides=[1,s,s,1],padding='SAME',name=name)


In [ ]:
#Losses and Accuracy

def loss(logits,labels):
    labels = tf.reshape(tf.cast(labels,tf.int64),[-1])
    #print labels.get_shape().as_list(),logits.get_shape().as_list()
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels,logits=logits,name='cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy,name='cross_entropy')
    total_loss = tf.add(tf.reduce_sum(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)),cross_entropy_mean,name='total_loss')
    return total_loss

def top_1_acc(logits,true_labels):
    pred_labels = tf.argmax(logits,1)
    true_labels = tf.cast(true_labels,tf.int64)
    #print pred_labels.get_shape().as_list(),true_labels
    correct_pred = tf.cast(tf.equal(pred_labels, true_labels), tf.float32)
    accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))
    return accuracy

def top_5_acc(logits,true_labels):
    true_labels = tf.cast(true_labels,tf.int64)
    return tf.reduce_mean(tf.cast(tf.nn.in_top_k(logits, true_labels, k=5,name='top_5_acc'), tf.float32))

In [ ]:
#Data Augmentation

def get_new_size():
    new_size = 96 + random.choice([24,16,0])
    return [new_size,new_size]

def get_random_augmentation_combinations(length):
    out = [False,True]
    return [random.choice(out) for i in range(length)]

In [ ]:
#Getting Image data

def get_all_images(img_file):
    images = np.fromfile(img_file,dtype=np.uint8).astype(np.float32)
    images = np.reshape(images,(-1,3,96,96))
    images = np.transpose(images,(0,3,2,1))
    print('Normalizing Inputs...')
    rmean = np.mean(images[:,:,:,0])
    gmean = np.mean(images[:,:,:,1])
    bmean = np.mean(images[:,:,:,2])
    images[:,:,:,0] = (images[:,:,:,0] - rmean)#/rstd
    images[:,:,:,1] = (images[:,:,:,1] - gmean)#/gstd
    images[:,:,:,2] = (images[:,:,:,2] - bmean)#/bstd
    print ('R_mean:',rmean,'G_mean:',gmean,'B_mean:',bmean)
    return images,rmean,gmean,bmean

def get_all_labels(label_file):
    labels = np.fromfile(label_file,dtype=np.uint8)
    #print labels.shape
    return labels

def get_test_images(img_file,rmean,gmean,bmean):
    images = np.fromfile(img_file,dtype=np.uint8).astype(np.float32)
    images = np.reshape(images,(-1,3,96,96))
    images = np.transpose(images,(0,3,2,1))
    print('Normalizing Validation Images...')
    images[:,:,:,0] = (images[:,:,:,0] - rmean)#/rstd
    images[:,:,:,1] = (images[:,:,:,1] - gmean)#/gstd
    images[:,:,:,2] = (images[:,:,:,2] - bmean)#/bstd
    return images

In [ ]:
#Create dataset
#Getting the dataset


print('Getting the data...')
train_data_path = '/floyd/input/stl10_binary/train_X.bin' #/media/siladittya/fdc481ce-9355-46a9-b381-9001613e3422/siladittya/StudyMaterials/ISI/code/ds/stl10_binary
train_label_path = '/floyd/input/stl10_binary/train_y.bin'

train_img_file = open(train_data_path,'rb')
train_label_file = open(train_label_path,'rb')

train_x,rmean,gmean,bmean = get_all_images(train_img_file)
train_y = get_all_labels(train_label_file)
train_y = train_y - 1
#Getting Validation Dataset
test_img_path = '/floyd/input/stl10_binary/test_X.bin'
test_label_path = '/floyd/input/stl10_binary/test_y.bin'

test_img_file = open(test_img_path,'rb')
test_label_file = open(test_label_path,'rb')

test_x = get_test_images(test_img_file,rmean,gmean,bmean)
test_y = get_all_labels(test_label_file)
test_y = test_y - 1
print('Getting Validation set from Test set...')
val_x = test_x[1:2000]
val_y = test_y[1:2000]

train_x = np.append(train_x,test_x[3000:],axis=0)
train_y = np.append(train_y,test_y[3000:],axis=0)
print("Training Set size = ",train_x.shape)

test_x = test_x[2000:3000]
test_y = test_y[2000:3000]

In [ ]:
#Placeholders and constants

index = np.arange(train_x.shape[0])

#........ This part will used to get training data for each epoch during training
num_epochs = 100
batch_size = 50
numiter = 200
ne = 0

feed_images = tf.placeholder(tf.float32,shape=(None,96,96,3))
feed_labels = tf.placeholder(tf.float32,shape=(None,))
lr = tf.placeholder(tf.float32,shape=())
keep_prob = tf.placeholder(tf.float32,shape=())
aug_img = tf.placeholder(tf.float32,shape=(96,96,3))

In [ ]:
#VGG 13 ConvNet

with tf.device('/gpu:0'):
    conv1 = conv2d(feed_images,'conv1',[3,3,3,64],1)
    conv2 = conv2d(conv1,'conv2',[3,3,64,64],1)
    pool1 = maxpool(conv2,'pool1',2,2)
    #size = [N,48,48,64]
    conv3 = conv2d(pool1,'conv3',[3,3,64,128],1)
    conv4 = conv2d(conv3,'conv4',[3,3,128,128],1)
    pool2 = maxpool(conv4,'pool2',2,2)
    #size = [N,24,24,128]
    conv5 = conv2d(pool2,'conv5',[3,3,128,256],1)
    conv6 = conv2d(conv5,'conv6',[3,3,256,256],1)
    pool3 = maxpool(conv6,'pool3',2,2)
    #size = [N,12,12,256]
    conv7 = conv2d(pool3,'conv7',[3,3,256,512],1)
    conv8 = conv2d(conv7,'conv8',[3,3,512,512],1)
    pool4 = maxpool(conv8,'pool4',2,2)
    #size = [N,6,6,512]
    conv9 = conv2d(pool4,'conv9',[3,3,512,512],1)
    conv10 = conv2d(conv9,'conv10',[3,3,512,512],1)
    pool5 = maxpool(conv10,'pool5',2,2)
    #size = [N,3,3,512]
    flattened_pool5 = tf.contrib.layers.flatten(pool5)
    fc1 = tf.contrib.layers.fully_connected(flattened_pool5,512,weights_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.001, dtype=tf.float32)))
    dropout1 = tf.nn.dropout(fc1,keep_prob)
    fc2 = tf.contrib.layers.fully_connected(dropout1,512,weights_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.001, dtype=tf.float32)))
    dropout2 = tf.nn.dropout(fc2,keep_prob)
    logits = tf.contrib.layers.fully_connected(dropout2,10,activation_fn=None,weights_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.001, dtype=tf.float32)))

    cost = loss(logits,feed_labels)

    opt_mom = tf.train.AdamOptimizer(learning_rate=0.0001)#,momentum=0.9)
    opt = opt_mom.minimize(cost)

    acc = top_1_acc(logits,feed_labels)
#Defined outside gpu0 device since tf.nn.in_top_k is not supported for gpu kernel
valacc = top_5_acc(logits,feed_labels)

In [ ]:
#Data Augmentation

img_scale_crop = tf.random_crop(tf.image.resize_images(aug_img,get_new_size()),[96,96,3])
img_rand_flip_lr = tf.image.random_flip_left_right(aug_img)
img_rand_flip_ud = tf.image.random_flip_up_down(aug_img)
img_con = tf.image.random_contrast(aug_img,0.01,0.25)
img_br = tf.image.random_brightness(aug_img,0.25)
img_rot = tf.contrib.image.rotate(aug_img,0.785*(-0.5+random.random()),interpolation='BILINEAR')

In [ ]:
#Training
ne = 0
val_x = val_x[:1000]
val_y = val_y[:1000]
sess = tf.Session()
sess.run(tf.global_variables_initializer())
tl=[]
vl=[]
ta=[] #Top-1 Training accuracy
ta5 = [] #Top-5 Training accuracy
va=[] #Top-1 Validation accuracy
va5 = [] #Top-5 Validation accuracy

while(ne<30):
    stime = time.time()
    print 'Epoch::',ne+1,'...'
    
    #Shuffling the Dataset
    if ne != 0:
        np.random.shuffle(index)
        train_x = train_x[index]
        train_y = train_y[index]
    
    for niter in range(numiter):
    
        if (niter+1)%50==0:
            print 'iter..',niter+1
        
        #Getting next Batch
        offset = niter*batch_size
        x_iter, y_iter = np.array(train_x[offset:offset+batch_size,:,:,:]), np.array(train_y[offset:offset+batch_size])
        
        #Data Augmentation
        for n in range(batch_size):
            args = get_random_augmentation_combinations(3)
            if args[0]:
                x_iter[n] = sess.run(img_scale_crop,feed_dict={aug_img:x_iter[n]})
            if args[1]:
                x_iter[n] = sess.run(img_rand_flip_lr,feed_dict={aug_img:x_iter[n]})
            if args[2]: 
                x_iter[n] = sess.run(img_rand_flip_ud,feed_dict={aug_img:x_iter[n]})
                
        feed_trdict={feed_images:x_iter,feed_labels:y_iter,keep_prob:0.55}#,lr:0.01
            
        #Train the optimizer
        sess.run(opt,feed_dict=feed_trdict)

    #Calculate accuracy of Training set
    cc = sess.run(cost,feed_dict=feed_trdict)
    tr_acc = sess.run(acc,feed_dict = {feed_images:x_iter,feed_labels:y_iter,keep_prob:1.0})
    top5_tr_acc = sess.run(valacc,feed_dict = {feed_images:x_iter,feed_labels:y_iter,keep_prob:1.0})
    ta.append(tr_acc)
    ta5.append(top5_tr_acc)
    tl.append(cc)
    
    #Calculate accuracy of Validation set
    val_loss = sess.run(cost,feed_dict = {feed_images:val_x,feed_labels:val_y,keep_prob:1.0})
    top5_val_acc = sess.run(valacc,feed_dict = {feed_images:val_x,feed_labels:val_y,keep_prob:1.0})
    top1_val_acc = sess.run(acc,feed_dict = {feed_images:val_x,feed_labels:val_y,keep_prob:1.0})
    va.append(top1_val_acc)
    va5.append(top5_val_acc)
    vl.append(val_loss)
    
    #print 'Epoch..',ne+1,'...'
    print 'Training accuracy-> Top-1::',tr_acc*100,'%','Top-5:: ',top5_tr_acc*100,'%',' Training cost::',cc
    print 'Top-1 Validation accuracy::',top1_val_acc*100,'Top-5 Val Accuracy:: ',top5_val_acc*100,'%',' Val loss: ',val_loss
    print 'Time reqd.::',(time.time()-stime)/60,'mins...'

    ne+=1

In [ ]:
builder.add_meta_graph_and_variables(sess, ["EVALUATING"])
builder.save()

In [ ]:
plt.plot(ta)
plt.plot(va)
plt.show()

In [ ]:
plt.plot(ta5)
plt.plot(va5)
plt.show()

In [ ]:
plt.plot(tl)
plt.plot(vl)
plt.show()

In [ ]:
import libmr as mr

In [ ]:
#---------------------------------META-RECOGNITION PART 1---------------------------------------#

#penult_layer = tf.random_uniform(shape=[1,10],maxval=1.0)
softmax_layer = tf.nn.softmax(logits)
pred_label = tf.argmax(softmax_layer,1)
_,top5_pred = tf.nn.top_k(softmax_layer, k=5)

mav = {}
av = {}

mav5 = {}
av5 = {}

for i in xrange(10):
    av[i] = np.array([]).reshape((0,10))

with tf.device('/gpu:0'):
    print 'Starting processing activation vectors...'
    stime = time.time()
    stime1=stime
    for i1 in xrange(10000):
        train_img = train_x[i1].reshape((1,96,96,3))
        true_label = train_y[i1]
        if (i1+1)%1000==0:
            print "Analysing","."*int(i1/1000)
            print 'Time for ',i1+1,': ',time.time()-stime,' secs...'
            stime = time.time()
        #For Top-1 classification
        penult,pred_y,top5pred = sess.run([logits,pred_label,top5_pred],feed_dict={feed_images:train_img,keep_prob:1.0})
        pred_y = pred_y[0]
        #stime = time.time()
        #print np.array(penult).shape
        
        if pred_y==true_label:
            av[true_label] = np.append(av[true_label],np.array(penult),axis=0)
            #print av[true_label].shape, pred_y, true_label
        #print'.........................', av
    print 'Completion time:: ', time.time()-stime1,'secs...'
    #Calculating the Mean activation Vector
    print "Calculating the Mean activation Vector..."
    for i in xrange(10):
        mav[i] = np.mean(av[i],axis=0)
        #print mav[i]
    
    dist_from_respmean = {}
    
    for i in xrange(10):
        dist_from_respmean[i] = []
    
    #dist_from_respmean5 = dist_from_respmean
    #Calculating the largest of the Distances
    print "Calculating the largest of the Distances..."
    for i in xrange(10):
        n = av[i].shape[0]
        #print n
        for j in xrange(n):
            penult = av[i][j]
            dist_from_respmean[i].append(np.sqrt(np.sum(np.square(np.subtract(penult,mav[true_label])))))


    #Sorting the distances
    print "Sorting the distances"
    for i in xrange(10):
        dist_from_respmean[i] = sorted(dist_from_respmean[i])
    
    
    #Weibull tail fitting
    models = []
    print "Weibull Tail Fitting..."
    for i in xrange(10):
        meta = mr.MR()
        meta.fit_high(dist_from_respmean[i][-50:],50)
        models.append(meta)


In [ ]:
#----------------------------META-RECOGNITION PART 2-------------------------------
for n in xrange(100,300):
    input_img = unlab_x[n].reshape((1,96,96,3))
    actvec = sess.run(logits,feed_dict = {feed_images:input_img,keep_prob:1.0})
    actvec = np.array(actvec[0])
    #print 'Activation Vector: ',actvec,'\n'
    #Calculating softmax values with unkown as 0.0
    softmax_val = np.exp(actvec)/np.sum(np.exp(actvec))
    softmax_val = np.insert(softmax_val,0,0.0)
    print 'Softmax Values: ',softmax_val
    #..........................Sorting the activation vector
    sorted_actvec = sorted(actvec)
    sorting_order = actvec.argsort()
    #print 'Sorted: ',sorted_actvec,'\n'
    #..................Calculatng alpha weight and dist from mean of each class
    alpha_weight = []
    dist_from_means = []
    for i in xrange(1,11):
        alpha_weight.append((11-i)/10.0)
        dist_from_means.append(np.sqrt(np.sum(np.square(np.subtract(actvec,mav[i-1])))))
    #print 'Distance from means:: ',dist_from_means,'\n'
    #..........................Calculating weilbull CDF score
    wscore = []
    #Unknown is taken as 0th label
    for i in xrange(1,11):
        wscore.append(1 - alpha_weight[i-1]*(1-models[i-1].w_score(dist_from_means[i-1])))
        
    #........................Calculating revised activation vector
    wscore = np.array(wscore)
    #print wscore
    revised_act = np.zeros((1,11))
    for i in xrange(1,11):
        revised_act[:,i] = actvec[i-1]*wscore[i-1]
    
    #Calculating activation vector value for 0th label i.e. unknown label
    revised_act[:,0] = np.sum(np.multiply(actvec,1 - wscore))
    #print revised_act
    #.........................Compute Openmax probabilities
    openmax_val = np.exp(revised_act)/np.sum(np.exp(revised_act))
    print 'Openmax valuess: ',openmax_val[0]
    #........................Checking the predictions
    top1_pred = np.argmax(np.array(openmax_val[0]))
    
    top5_preds = np.array(openmax_val[0]).argsort()[-5:][::-1]
    
    plt.imshow((unlab_x[n]+np.reshape(np.array([rmean,gmean,bmean]),(1,1,3)))/255)
    plt.show()
    print '0->Unknown|1->Aeroplane|2->Bird|3->Car|4->Cat|5->Deer|6->Dog|7->Horse|8->Monkey|9->Ship|10->Truck'
    if top1_pred == 0 or openmax_val[0][top1_pred] < 0.5:
        print 'Prediction: Unknown\n'
        print 'Unrevised Values: ',np.array(softmax_val).argsort()[-5:][::-1]
        print 'Revised Top 5:' ,top5_preds
    else:
        print 'Prediction: Known'
        print 'Unrevised Values: ',np.array(softmax_val).argsort()[-5:][::-1]
        print 'Revised Top 5:' ,top5_preds
    print '..........................................................................................'